In [121]:
import sys
import os
# Get the current directory and parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
sys.path.insert(0, current_dir)

# Import from the-machine module
import importlib.util
the_machine_path = os.path.join(parent_dir, "the-machine")
sys.path.insert(0, the_machine_path)

# Import all from the-machine and custom_matroid_functions
from HM_module import *
from custom_matroid_functions import *

In [122]:
P = LaurentPolynomialRing(ZZ, 'x')
x = P.gen()
n = 5
M = matroids.CompleteGraphic(n)
r = M.rank()
rank_func = lambda x : M.rank(x)

#e = next(x for x in M.groundset() if x  not in M.coloops())
e = list(M.groundset())[0]
print(M.groundset())
print("deleting ", e)
mdele = M.delete(e)
Eq_M = create_deformed_algebra(M, base_ring=P)
Eq_mdele = create_deformed_algebra(mdele, base_ring=P)
E_mdele = Eq_mdele.natural()

E = Eq_M.natural()
E_basis = E.basis()
E_mdele_basis = E_mdele.basis()

C = Eq_M.C()
C_dele = Eq_mdele.C()
C_basis = C.basis()
C_dele_basis = C_dele.basis()

Z = Eq_M.zeta()
Z_basis = Z.basis()

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})
deleting  0


In [123]:
print("rank of M: ", r)
print("chow polynomial of M: ", chow_polynomial(M))

rank of M:  4
chow polynomial of M:  x^3 + 41*x^2 + 41*x + 1


In [124]:
rev = {value: key for key, value in E_basis.items()}

def delta_flat(F):
    F_del_e = frozenset(set(F) - {e})
    return Eq_mdele(x^( - M.rank(F) + mdele.rank(F_del_e)) * E_mdele_basis[F_del_e])
    
def delta(elem):
    rep = E(elem)
    decom = {monomial: coeff for monomial, coeff in zip(rep.monomials(), rep.coefficients())}
    return Eq_mdele(sum(decom[F] * delta_flat(rev[F]) for F in decom))

dict = {F : C_dele(delta(C[F])) for F in E_basis.keys()}
dict

{frozenset(): C[frozenset()],
 frozenset({1}): C[frozenset({1})],
 frozenset({8}): C[frozenset({8})],
 frozenset({9}): C[frozenset({9})],
 frozenset({1, 9}): C[frozenset({1, 9})],
 frozenset({5}): C[frozenset({5})],
 frozenset({1, 5}): C[frozenset({1, 5})],
 frozenset({5, 8}): C[frozenset({8, 5})],
 frozenset({4}): C[frozenset({4})],
 frozenset({4, 9}): C[frozenset({9, 4})],
 frozenset({2}): C[frozenset({2})],
 frozenset({2, 8}): C[frozenset({8, 2})],
 frozenset({2, 4}): C[frozenset({2, 4})],
 frozenset({7}): C[frozenset({7})],
 frozenset({1, 2, 7}): C[frozenset({1, 2, 7})],
 frozenset({4, 5, 7}): C[frozenset({4, 5, 7})],
 frozenset({7, 8, 9}): C[frozenset({8, 9, 7})],
 frozenset({3}): C[frozenset({3})],
 frozenset({1, 3, 8}): C[frozenset({8, 1, 3})],
 frozenset({2, 3, 9}): C[frozenset({9, 2, 3})],
 frozenset({3, 4}): C[frozenset({3, 4})],
 frozenset({2, 3, 4, 9}): C[frozenset({9, 2, 3, 4})],
 frozenset({3, 7}): C[frozenset({3, 7})],
 frozenset({1, 2, 3, 7, 8, 9}): C[frozenset({1, 2, 3

In [125]:
def flat_division(M, e):
    L = M.lattice_of_flats()
    flat_div = {'S':set(), 'T':set(), 'A':set(), 'B':set()}
    for F in L:
        if e in F:
            if F - {e} in L:
                flat_div['T'].add(F)
            else:
                flat_div['A'].add(F)
        else:
            if F.union({e}) in L:
                flat_div['S'].add(F)
            else:
                flat_div['B'].add(F)
    return flat_div

print(flat_division(M, e))

{'S': {frozenset({9}), frozenset({8}), frozenset({7}), frozenset(), frozenset({8, 9, 7})}, 'T': {frozenset({0, 8, 9, 7}), frozenset({0}), frozenset({0, 7}), frozenset({0, 8}), frozenset({0, 9})}, 'A': {frozenset({0, 1, 4}), frozenset({0, 3, 6}), frozenset({0, 2, 3, 5, 6, 9}), frozenset({0, 2, 5}), frozenset({0, 1, 4, 9}), frozenset({0, 3, 6, 7}), frozenset({0, 8, 2, 5}), frozenset({0, 1, 2, 4, 5, 7}), frozenset({0, 1, 3, 4, 6, 8}), frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})}, 'B': {frozenset({8, 2}), frozenset({2, 6}), frozenset({1}), frozenset({1, 2, 6, 7}), frozenset({9, 4}), frozenset({9, 2, 3, 4}), frozenset({3, 4}), frozenset({9, 2, 3}), frozenset({1, 2, 7}), frozenset({2, 4}), frozenset({9, 5, 6}), frozenset({3, 5}), frozenset({8, 1, 3, 5}), frozenset({8, 5}), frozenset({1, 6}), frozenset({1, 2, 3, 7, 8, 9}), frozenset({8, 2, 4, 6}), frozenset({3, 7}), frozenset({6, 7}), frozenset({1, 5}), frozenset({8, 4, 6}), frozenset({5}), frozenset({3, 4, 5, 7}), frozenset({4, 5, 7}), frozens

In [126]:
from IPython.display import HTML, display
import pandas as pd

# Get the flat division
flat_div = flat_division(M, e)

# Create separate data lists for each category
data_s = []
data_t = []
data_a = []
data_b = []

basis_list_mdele = list(mdele.lattice_of_flats())

for F in dict:
    elem = dict[F]
    coeff_empty = elem.monomial_coefficients().get(frozenset(), 0)
    
    terms = []
    for S, coeff in elem.monomial_coefficients().items():
        terms.append(f"({coeff})*C[{str(set(S))}]")
    formatted_expr = " + ".join(terms) if terms else "0"
    
    row_data = {
        'F': str(set(F)),
        'delta(C[F])': formatted_expr,
        'coeff of empty': coeff_empty
    }
    
    # Categorize the flat and add to appropriate list
    if F in flat_div['S']:
        data_s.append(row_data)
    elif F in flat_div['T']:
        data_t.append(row_data)
    elif F in flat_div['A']:
        data_a.append(row_data)
    elif F in flat_div['B']:
        data_b.append(row_data)

# Create and display tables for each category
def display_table(data, category_name, color='lightblue'):
    if data:
        df = pd.DataFrame(data)
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} ({len(data)} flats)</h3>'))
        display(HTML(
            f'<div style="height: 300px; overflow-y: auto; border: 2px solid {color}; padding: 10px;">{df.to_html(index=False)}</div>'
        ))
    else:
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} (0 flats)</h3>'))
        display(HTML('<p>No flats in this category.</p>'))

# Display all four tables
display_table(data_s, 'S', 'green')
display_table(data_t, 'T', 'coral') 
display_table(data_a, 'A', 'blue')
display_table(data_b, 'B', 'red')

F,delta(C[F]),coeff of empty
set(),(1)*C[set()],1
{8},(1)*C[{8}],0
{9},(1)*C[{9}],0
{7},(1)*C[{7}],0
"{8, 9, 7}","(1)*C[{8, 9, 7}]",0


F,delta(C[F]),coeff of empty
{0},(x^-1 - x)*C[set()],x^-1 - x
"{0, 7}",(-1)*C[set()] + (x^-1 - x)*C[{7}],-1
"{0, 8}",(-1)*C[set()] + (x^-1 - x)*C[{8}],-1
"{0, 9}",(-1)*C[set()] + (x^-1 - x)*C[{9}],-1
"{0, 8, 9, 7}","(-x^-1 - x)*C[set()] + (-1)*C[{8}] + (-1)*C[{9}] + (-1)*C[{7}] + (x^-1 - x)*C[{8, 9, 7}]",-x^-1 - x


F,delta(C[F]),coeff of empty
"{0, 1, 4}","(-1 + x^2)*C[set()] + (1)*C[{1, 4}]",-1 + x^2
"{0, 3, 6}","(-1 + x^2)*C[set()] + (1)*C[{3, 6}]",-1 + x^2
"{0, 2, 5}","(-1 + x^2)*C[set()] + (1)*C[{2, 5}]",-1 + x^2
"{0, 1, 2, 4, 5, 7}","(-x^-1 + 2*x - x^3)*C[set()] + (-1 + x^2)*C[{7}] + (1)*C[{1, 2, 4, 5, 7}]",-x^-1 + 2*x - x^3
"{0, 3, 6, 7}","(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{7}] + (1)*C[{3, 6, 7}]",-x^-1 + x
"{0, 1, 3, 4, 6, 8}","(-x^-1 + 2*x - x^3)*C[set()] + (-1 + x^2)*C[{8}] + (1)*C[{1, 3, 4, 6, 8}]",-x^-1 + 2*x - x^3
"{0, 8, 2, 5}","(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{8}] + (1)*C[{8, 2, 5}]",-x^-1 + x
"{0, 2, 3, 5, 6, 9}","(-x^-1 + 2*x - x^3)*C[set()] + (-1 + x^2)*C[{9}] + (1)*C[{2, 3, 5, 6, 9}]",-x^-1 + 2*x - x^3
"{0, 1, 4, 9}","(-x^-1 + x)*C[set()] + (-1 + x^2)*C[{9}] + (1)*C[{1, 4, 9}]",-x^-1 + x
"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}","(-x^-2 + 2 - 2*x^2 + x^4)*C[set()] + (-x^-1 + 2*x - x^3)*C[{8}] + (-x^-1 + 2*x - x^3)*C[{9}] + (-x^-1 + 2*x - x^3)*C[{7}] + (-1 + x^2)*C[{8, 9, 7}] + (1)*C[{1, 2, 3, 4, 5, 6, 7, 8, 9}]",-x^-2 + 2 - 2*x^2 + x^4


F,delta(C[F]),coeff of empty
{1},(1)*C[{1}],0
"{1, 9}","(1)*C[{1, 9}]",0
{5},(1)*C[{5}],0
"{1, 5}","(1)*C[{1, 5}]",0
"{8, 5}","(1)*C[{8, 5}]",0
{4},(1)*C[{4}],0
"{9, 4}","(1)*C[{9, 4}]",0
{2},(1)*C[{2}],0
"{8, 2}","(1)*C[{8, 2}]",0
"{2, 4}","(1)*C[{2, 4}]",0
